In [101]:
%pip install pfapack

Note: you may need to restart the kernel to use updated packages.


In [102]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
from pfapack import pfaffian
import math

In [103]:
def create_u_matrix(N1, N2, bc1, bc2):
    N = N1 * N2 # N1 为 a_1 方向 unit cell 数量
    M = np.zeros((2 * N, 2 * N)) # M 储存 u_jk(j 属于 A 子格，k 属于 B 子格)构型
    u = np.zeros((2 * N, 2 * N))
    
    if bc1 == "PBC":
        sign1 = 1
    elif bc1 == "APBC":
        sign1 = -1
    if bc2 == "PBC":
        sign2 = 1
    elif bc2 == "APBC":
        sign2 = -1    
    
    def index(n1, n2, sub):
        return n1 + n2 * N1 + N1 * N2 * sub # A子格对应 sub=0,B子格对应 sub=1
    
    for n2 in range(N2):
        for n1 in range(N1):
            j = index(n1, n2, 0) # j 为(n1, n2)unit cell 内 A 子格格点的单指标
            
            kx = index(n1, n2, 1) # kx 为(n1, n2)unit cell 内 B 子格格点的单指标
            M[j, kx] = 1 # j 与 kx 由 x-bond 相连，一定不跨越边界
            
            ky = index((n1 + 1) % N1, n2, 1) # ky 为 (n1, n2) 往 a1 方向走一步得到的 unit cell 内 B 子格格点的单指标j 与 ky 由 y-bond 相连
            if n1 == N1 - 1:
                M[j, ky] = sign1
            else:
                M[j, ky] = 1

            kz = index(n1, (n2 + 1) % N2, 1) # kz 为 (n1, n2) 往 a2 方向走一步得到的 unit cell 内 B 子格格点的单指标，j 与 kz 由 z-bond 相连
            if n2 == N2 - 1:
                M[j, kz] = sign2
            else:
                M[j, kz] = 1
      
    u = M - M.T                        
    return u

In [104]:
def create_t_matrix(N1, N2, Kx, Ky, Kz, u):
    N = N1 * N2
    t = np.zeros((N, N))
    K = np.array([Kx, Ky, Kz])
    list = np.array([[0, 0], [1, 0], [0, 1]])
    for i in range(N):
        for j in range(N):
            sum = 0
            r_i_vec = np.array([i % N1, i // N1])
            r_j_vec = np.array([j % N1, j // N1])
            
            for delta_vec in list:
                tmp1 = (r_j_vec[0] - delta_vec[0] + N1) % N1
                tmp2 = (r_j_vec[1] - delta_vec[1] + N2) % N2
                if (np.array_equal(r_i_vec,np.array([tmp1, tmp2]))):
                    delta_func = 1
                else:
                    delta_func = 0
                    
                tmp1 = delta_vec[0] + delta_vec[1] * 2
                K_delta = K[tmp1]

                sum += delta_func * K_delta
            sum *= u[i, j + N]
            t[i, j] = sum
    return t


In [105]:
def create_t_plus_matrix(N1, N2, u):
    N = N1 * N2
    t_plus = np.zeros((N, N))
    def index1(UC_vec, sub):
        return N1 * UC_vec[1] + UC_vec[0] + sub * N
    list1 = [np.array([1, 0]), np.array([0, 1]), np.array([0, 0])]
    list2 = [np.array([0, 1]), np.array([0, 0]), np.array([1, 0])]
    for i in range(N):
        for j in range(N):
            r_i_vec = np.array([i % N1, i // N1])
            r_j_vec = np.array([j % N1, j // N1])
            sum = 0
            
            for delta1_vec, delta2_vec in zip(list1, list2):
                tmp1 = (r_i_vec[0]-delta1_vec[0] + N1) % N1 # r_i - delta_1 的 a_1 分量，考虑了周期性边界条件
                tmp2 = (r_i_vec[1]-delta1_vec[1] + N2) % N2 # r_i - delta_1 的 a_2 分量，考虑了周期性边界条件
                tmp3 = (r_j_vec[0]-delta2_vec[0] + N1) % N1 # r_j - delta_2 的 a_1 分量，考虑了周期性边界条件
                tmp4 = (r_j_vec[1]-delta2_vec[1] + N2) % N2 # r_j - delta_2 的 a_2 分量，考虑了周期性边界条件
                if (np.array_equal(np.array([tmp1, tmp2]) , np.array([tmp3, tmp4]))):
                    delta_func = 1
                else:
                    delta_func = 0
                sum += delta_func * u[index1(r_i_vec, 1), index1(np.array([tmp1, tmp2]), 0) ] * u[index1(r_j_vec, 1), index1(np.array([tmp3, tmp4]), 0)]

            t_plus[i][j] = sum
            
    return t_plus

In [106]:
def create_t_minus_matrix(N1, N2, u):
    N = N1 * N2
    t_minus = np.zeros((N, N))
    def index1(UC_vec, sub):
        return N1 * UC_vec[1] + UC_vec[0] + sub * N
    list1 = [np.array([-1, 0]), np.array([0, -1]), np.array([0, 0])]
    list2 = [np.array([0, -1]), np.array([0, 0]), np.array([-1, 0])]
    for i in range(N):
        for j in range(N):
            r_i_vec = np.array([i % N1, i // N1])
            r_j_vec = np.array([j % N1, j // N1])
            sum = 0
            
            for delta1_vec, delta2_vec in zip(list1, list2):
                tmp1 = (r_i_vec[0]-delta1_vec[0] ) % N1 # r_i - delta_1 的 a_1 分量，考虑了周期性边界条件
                tmp2 = (r_i_vec[1]-delta1_vec[1] ) % N2 # r_i - delta_1 的 a_2 分量，考虑了周期性边界条件
                tmp3 = (r_j_vec[0]-delta2_vec[0] ) % N1 # r_j - delta_2 的 a_1 分量，考虑了周期性边界条件
                tmp4 = (r_j_vec[1]-delta2_vec[1] ) % N2 # r_j - delta_2 的 a_2 分量，考虑了周期性边界条件
                if (np.array_equal(np.array([tmp1, tmp2]) , np.array([tmp3, tmp4]))):
                    delta_func = 1
                else:
                    delta_func = 0
                    
                sum += delta_func * u[index1(r_i_vec, 0), index1(np.array([tmp1, tmp2]), 1) ] * u[index1(r_j_vec, 0), index1(np.array([tmp3, tmp4]), 1)]

            t_minus[i][j] = sum
            
    return t_minus

In [107]:
def create_P_times_P_prime_matrix(N):
    P = np.zeros((2 * N, 2* N))
    for i in range(N):
        P[i, 2 * i] = 1
        P[i + N, 2 * i + 1] = 1
    # 计算 P_prime 矩阵
    I_N = np.eye(N) # N \times N 单位矩阵
    P_prime = np.kron(I_N, 0.5 * np.array([[1, 1j], [1, -1j]]))
    #print("P_prime:\n", P_prime)
    P_times_P_prime = np.dot(P, P_prime) # 矩阵乘法
    return P_times_P_prime

In [108]:
N1, N2, Kx, Ky, Kz, kappa = 10, 10, 1, 1, 1, 0.2
N = N1 * N2

In [109]:
results_H_0 = {} # 空字典，以 (bc1, bc2) 为键，每个键储存对应边界条件下 H0 的本征值、本征矢量、基态能量
results_H_total = {} # 空字典，以 (bc1, bc2) 为键，每个键储存对应边界条件下 H_total = H0 + H_kappa 的本征值、本征矢量、基态能量
M_0 = np.zeros((2 * N, 2 * N))
M_kappa = np.zeros((2 * N, 2 * N))
P_times_P_prime = create_P_times_P_prime_matrix(N)

for bc1 in ["PBC", "APBC"]:
    for bc2 in ["PBC", "APBC"]:
        u = create_u_matrix(N1, N2, bc1, bc2)
        t = create_t_matrix(N1, N2, Kx, Ky, Kz, u)
        #print("t_matrix:\n", t)
        t_plus = create_t_plus_matrix(N1, N2, u)
        t_minus = create_t_minus_matrix(N1, N2, u)
        for i in range(N):
            for j in range(N):
                M_0[2 * i, 2 * j] = 0
                M_0[2 * i, 2 * j + 1] = t[i, j]
                M_0[2 * i + 1, 2 * j] = -t[j, i]
                M_0[2 * i + 1, 2 * j + 1] = 0
                M_kappa[2 * i, 2 * j] = kappa * (t_minus[i, j] - t_minus[j, i])
                M_kappa[2 * i, 2 * j + 1] = 0
                M_kappa[2 * i + 1, 2 * j] = 0
                M_kappa[2 * i + 1, 2 * j + 1] = kappa * (t_plus[i, j] - t_plus[j, i])

        #print("M_0:", M_0)
        #print("M_kappa:", M_kappa)
        pf = pfaffian.pfaffian(M_0)
        parity = np.sign(pf)
        H_0 = np.zeros((2 * N, 2 * N))
        H_0 = 1j * np.linalg.multi_dot([np.linalg.inv(P_times_P_prime).T.conj(), M_0, np.linalg.inv(P_times_P_prime)])
        eigvals, eigvecs = np.linalg.eigh(H_0)
        ground_state_energy = np.sum(eigvals[:N]) / 2 # 计算无磁场时基态能量
        #print("ground_state_energy:", ground_state_energy)
        results_H_0[(bc1, bc2)] = {
            "eigvals": eigvals,
            "eigvecs": eigvecs,
            "ground_state_energy": ground_state_energy,
            "pf": pf,
            "parity": parity
        }


        
        M = M_0 + M_kappa
        pf = pfaffian.pfaffian(M)
        parity = np.sign(pf)
        H_total = np.zeros((2 * N, 2 * N))
        H_total = 1j * np.linalg.multi_dot([np.linalg.inv(P_times_P_prime).T.conj(), M, np.linalg.inv(P_times_P_prime)])
        eigvals, eigvecs = np.linalg.eigh(H_total)
        ground_state_energy = np.sum(eigvals[:N]) / 2 
        #print("ground_state_energy:", ground_state_energy)
        results_H_total[(bc1, bc2)] = {
            "eigvals": eigvals,
            "eigvecs": eigvecs,
            "ground_state_energy": ground_state_energy,
            "pf": pf,
            "parity": parity
        }
        #print(bc1, bc2, pf, parity)
print(f"config:N1={N1},N2={N2},Kx={Kx},Ky={Ky},Kz={Kz},kappa={kappa}\n")
for key in results_H_0:
    print(f"(bc1, bc2)={key}:")
    print(f"H_0:ground_state_energy = {results_H_0[key]['ground_state_energy']}", )
    print(f"H_0:pf = {results_H_0[key]['pf']:}")
    print(f"H_0:parity = {results_H_0[key]['parity']}\n")
for key in results_H_total:
    print(f"(bc1, bc2)={key}:")
    print(f"H_total:ground_state_energy = {results_H_total[key]['ground_state_energy']}")
    print(f"H_total:pf = {results_H_total[key]['pf']:}", )
    print(f"H_total:parity = {results_H_total[key]['parity']}\n", )

config:N1=10,N2=10,Kx=1,Ky=1,Kz=1,kappa=0.2

(bc1, bc2)=('PBC', 'PBC'):
H_0:ground_state_energy = -157.53598668311992
H_0:pf = -210736858987744.25
H_0:parity = -1.0

(bc1, bc2)=('PBC', 'APBC'):
H_0:ground_state_energy = -157.44132763536777
H_0:pf = 100177750390624.6
H_0:parity = 1.0

(bc1, bc2)=('APBC', 'PBC'):
H_0:ground_state_energy = -157.44132763536777
H_0:pf = 100177750390528.39
H_0:parity = 1.0

(bc1, bc2)=('APBC', 'APBC'):
H_0:ground_state_energy = -157.44132763536774
H_0:pf = 100177750390528.7
H_0:parity = 1.0

(bc1, bc2)=('PBC', 'PBC'):
H_total:ground_state_energy = -169.8008983400607
H_total:pf = -2.5831759277984894e+20
H_total:parity = -1.0

(bc1, bc2)=('PBC', 'APBC'):
H_total:ground_state_energy = -169.8009704066353
H_total:pf = 2.5837281190942048e+20
H_total:parity = 1.0

(bc1, bc2)=('APBC', 'PBC'):
H_total:ground_state_energy = -169.80097040663526
H_total:pf = 2.5837281190941993e+20
H_total:parity = 1.0

(bc1, bc2)=('APBC', 'APBC'):
H_total:ground_state_energy = -169.8009